# Exploratory Analysis

In [3]:
# Imports
import requests
import json
import yfinance as yf
import numpy as np
from openai import OpenAI
from newsapi import NewsApiClient 

In [7]:
print("\nProject AI Investing - Using LLMs for Investment Analytics")

# Setting the secret keys
secret_key = '../ignore/keys.json'

# Open and read Json file
with open(secret_key, 'r', encoding='utf-8') as file:
    key = json.load(file)



client = OpenAI(api_key = key['gpt'])
newsapi = NewsApiClient(api_key = key['news_api'])


Project AI Investing - Using LLMs for Investment Analytics


In [36]:
# Function to get news about a specific company
def collect_news(company_name):
    # Makes a request to the NewsAPI to get articles about the company, in Portuguese and sorted by relevance
    all_articles = newsapi.get_everything(q = company_name,
    language = 'en',
    sort_by = 'relevancy',
    page_size = 20)

    # Initializes a list to store the news titles
    news_list = []

    # Goes through all the articles returned by the API
    for article in all_articles['articles']:
        # Extracts the title of each article
        title = article['title']

        # Checks if the title is not empty and if it does not contain '[Removed]'
        if title and '[Removed]' not in title:
            # Adds the valid title to the news list
            news_list.append(title)

    # Returns the list of news titles
    return news_list

In [37]:
# Function to get stock data using the yfinance library
def collect_stock_data(ticker):

    # Creates a Ticker object for the specified stock
    stock = yf.Ticker(ticker)

    # Gets historical stock data for the last month
    data = stock.history(period='1mo')

    # Returns historical stock data
    return data

In [38]:
# Function to analyze the sentiment of a list of news items
def analyze_sentiment(company_name, news_list):

    # Initialize a list to store the sentiments of the news items
    sentiments = []

    # Loop through each news item in the news item list
    for news in news_list:

        # Set a prompt for the GPT model to analyze the sentiment of the news item title
        prompt = f"Classify the sentiment of the following news item title about {company_name} as 'positive', 'negative', or 'neutral': \"{news}\""

        # Send the prompt to the AI ​​model and get the response
        response = client.chat.completions.create(model = "gpt-4o",
        messages = [
            {"role": "user", "content": prompt}],
            temperature = 0.2)

        # Extract the content of the response and convert it to lowercase
        sentiment = response.choices[0].message.content.strip().lower()

        # Check if sentiment is 'positive', 'negative' or 'neutral' and adds it to the list
        if sentiment in ['positive', 'negative', 'neutral']:
            sentiments.append(sentiment)
        else:
            # Add 'neutral' if sentiment is not identified
            sentiments.append('neutral')

    # Returns the list of sentiments
    return sentiments

In [39]:
# Function to convert text sentiments to numeric values
def convert_sentiment_to_numerical(sentiments):

    # Initialize a list to store numeric sentiment scores
    sentiment_scores = []

    # Loop through each sentiment in the sentiment list
    for sentiment in sentiments:

        # Assign 1 for 'positive'
        if 'positive' in sentiment:
            sentiment_scores.append(1)

        # Assign -1 for 'negative'
        elif 'negative' in sentiment:
            sentiment_scores.append(-1)

        # Assign 0 for 'neutral'
        else:
            sentiment_scores.append(0)

    # Returns the list of numeric scores
    return sentiment_scores

In [40]:
# Function to make a decision based on stock data and sentiment
def make_decision(stock_data, sentiment_scores):

    # Calculates the daily percentage change in the stock's closing price
    stock_data['Pct_Change'] = stock_data['Close'].pct_change()

    # Get the most recent percentage change
    recent_change = stock_data['Pct_Change'].iloc[-1]

    # Calculates the average of sentiments if there are values ​​in the list
    if sentiment_scores:
        avg_sentiment = np.mean(sentiment_scores)
    else:
        # Sets 0 as default value if the list is empty
        avg_sentiment = 0

    # Sets the decision based on sentiment values ​​and price change
    if avg_sentiment > 0 and recent_change > 0:
        decision = 'Buy'
    elif avg_sentiment < 0 and recent_change < 0:
        decision = 'Sell'
    else:
        decision = 'Keep'

    # Returns the final decision
    return decision

In [41]:
input = {
    "company_name":'Apple',
    "ticker":'AAPL'
}


# Get the news about Apple
news = collect_news(input["company_name"])

# Display the number of news items retrieved and the list of news items
print(f"\nNumber of news items retrieved: {len(news)}")

print("\nThe AI ​​Investing Robot is Working. Please wait!\n")


Number of news items retrieved: 6

The AI ​​Investing Robot is Working. Please wait!



In [42]:
# Check for news to analyze
if not news:
    # Set 'Keep' as default decision if there is no news
    print("No news was retrieved.")
    decision = 'Keep'

else:
    # Analyze the sentiment of the news
    sentiments = analyze_sentiment(company_name=input["company_name"], news_list=news)

    # Display the analyzed sentiments
    for i in range(len(sentiments)):
        print("Sentiment: {} \t {}".format(sentiments[i], news[i]))

    # Convert the sentiments into numeric values
    sentiment_scores = convert_sentiment_to_numerical(sentiments)

    # Make a decision based on the stock data and sentiments
    decision = make_decision(collect_stock_data(input["ticker"]), sentiment_scores)

    # Display the final decision
    print(f"\nInvestment Decision: {decision}")

    print("\nThanks for Using Sentiment Stocks AI - Come Back Soon!\n")

Sentiment: neutral 	 How to Use Apple’s Genmoji to Create New Kinds of Emojis
Sentiment: positive 	 Netflix finally decides to play nice and support the Apple TV app
Sentiment: positive 	 The 4.8-Star-Rated 2024 15-Inch Apple MacBook Air Is $250 Off on Amazon for MLK Weekend
Sentiment: neutral 	 Apple is pausing notification summaries for news in the latest iOS 18.3 beta
Sentiment: neutral 	 I’m Obsessed With J.D. Vance’s Apple Watch
Sentiment: neutral 	 Samsung borrows from the Apple Wallet playbook with layaway and tap-to-send for Wallet

Investment Decision: Buy

Thanks for Using Sentiment Stocks AI - Come Back Soon!

